In [12]:
from sqlalchemy import create_engine
import pandas as pd



In [13]:
# connect to the postgres db we created containing the transaction data using sqlalchemy
# create a SQLAlchemy engine
engine = create_engine('postgresql://djschor:test/9@localhost:5408/ramp')

# input the query solution for  January 31's rolling 3 day average of total transaction amount processed per day
query = """
WITH daily_total AS (
    SELECT
        DATE(transaction_time) AS day,
        SUM(transaction_amount) AS total_amount
    FROM transactions
    WHERE DATE(transaction_time) BETWEEN '2021-01-29' AND '2021-01-31'
    GROUP BY day
),
rolling_avg AS (
    SELECT
        day,
        AVG(total_amount) OVER (
            ORDER BY day
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) AS rolling_average
    FROM daily_total
)
SELECT
    day,
    rolling_average
FROM rolling_avg
WHERE day = '2021-01-31';
"""

# load query results into a Pandas df
rolling_avg = pd.read_sql(query, engine)


In [14]:
# look at results
rolling_avg

,day,rolling_average
0,2021-01-31,682.15


In [15]:
# close the database connection
conn.close()
